In [ ]:
#    GNU LESSER GENERAL PUBLIC LICENSE
#    Version 3, 29 June 2007
#    Copyright (C) 2007 Free Software Foundation, Inc. <http://fsf.org/>
#    Everyone is permitted to copy and distribute verbatim copies
#    of this license document, but changing it is not allowed.

#    James Gaboardi, 2016

## Spatial Optimization Model Building in Python: Cplex v. Gurobi
## *p*-Dispersion Problem Holding *p* Constant 
## Increasing Matrix Dimensions from 5x5 to 100x100

----

#### James D. Gaboardi &nbsp;&nbsp; |  &nbsp;&nbsp; Florida State University &nbsp; &nbsp;|  &nbsp;&nbsp; Department of Geography 

----

#KUBY

In [1]:
# Imports
import pysal as ps
import geopandas as gpd
import numpy as np
from collections import OrderedDict
import pandas as pd
import qgrid
import cplex as cp
import gurobipy as gbp
import time
import bokeh
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook, output_file, show
from bokeh.models import (HoverTool, BoxAnnotation, GeoJSONDataSource, 
                          GMapPlot, GMapOptions, ColumnDataSource, Circle, 
                          DataRange1d, PanTool, WheelZoomTool, BoxSelectTool,
                          ResetTool, MultiLine)
output_notebook()

Loading BokehJS ...

In [8]:
N = 96  # Matrix Dimensions

# PANDAS DATAFRAME OF p/y results
n_list = []
for i in range(5, N+6):
    matrix = 'n='+str(i)+'x'+str(i)
    n_list.append(matrix)
y_list = []
for i in range(1, N+1):
    y = 'y'+str(i)
    y_list.append(y)

In [9]:
solve_df = pd.DataFrame(index=n_list)

In [10]:
solve_df

""
n=5x5
n=6x6
n=7x7
n=8x8
n=9x9
n=10x10
n=11x11
n=12x12
n=13x13
n=14x14


In [ ]:
# PDP
VAL_DISPERSION_cplex = []
VAL_DISPERSION_gurobi = []
solve_time_cplex = []
solve_time_gurobi = []

In [ ]:
def Cplex_and_Gurobi_pDispersion(dij, p_facilities, total_facilities):
    
    t1CPLEX = time.time()
    
    m = cp.Cplex()                                      # Instantiate a model
    m.parameters.emphasis.mip.set(2)                    # Set MIP emphasis to Optimal
    m.set_problem_type(m.problem_type.LP)               # Set problem type
    m.objective.set_sense(m.objective.sense.maximize)   # Objective Function ==>  Maximize

    # Service Nodes
    service_nodes = range(total_facilities)
    
    # Max Value in dij
    M_cplex = np.amax(dij)
    
    #  Add Variables        
    facility_variable_CPLEX = []
    for destination in service_nodes:
        facility_variable_CPLEX.append([])
        facility_variable_CPLEX[destination].append('y' + str(destination+1))
    
    # Add Maximized Minimum Variable
    D_cplex = 'D'
    m.variables.add(names=D_cplex,
                      obj=[1],
                       lb=[0],
                       ub=[cp.infinity],
                    types=['C'])
    
    # Add Facility Decision Variables
    m.variables.add(names=[facility_variable_CPLEX[destination][0] for destination in service_nodes],
                       lb=[0]*total_facilities, 
                       ub=[1]*total_facilities, 
                    types=['B']*total_facilities)
    
    # Add Facility Constraint
    facility_constraint_CPLEX = cp.SparsePair(ind=[facility_variable_CPLEX[destination][0] 
                                                                for destination in service_nodes], 
                                        val=[1.0] * total_facilities)
    m.linear_constraints.add(lin_expr=[facility_constraint_CPLEX],
                               senses=['E'],
                                  rhs=[p_facilities])
    
    # Add Inter-Facility Distance Constraints ==> n(n-1)/2
    index_value_rhs = [[],[],[]]
    for origin in service_nodes:
        for destination in service_nodes:
            if destination > origin:
                        index_value_rhs[0].append([facility_variable_CPLEX[origin][0]]+        \
                                                  [facility_variable_CPLEX[destination][0]]+[D_cplex])
                        index_value_rhs[1].append([-M_cplex]+[-M_cplex]+[-1.0])
                        index_value_rhs[2].append(-2*M_cplex-dij[origin][destination])
            else:
                pass

    number_of_constraints = range(len(index_value_rhs[0]))
    for record in number_of_constraints:
        inter_facility_constraints = index_value_rhs[0][record],                       \
                                     index_value_rhs[1][record]
        m.linear_constraints.add(lin_expr=[inter_facility_constraints],                 
                                   senses=['G'], 
                                      rhs=[index_value_rhs[2][record]])

    # Optimize and Print Results
    m.write('path.lp')
    m.solve()
    solution = m.solution
    t2CPLEX = round(round(time.time()-t1CPLEX, 3)/60, 5)
    
    selected_PDP_cplex = OrderedDict()
    for f in facility_variable_CPLEX:
        if 'x' in f[0]:
            pass
        elif solution.get_values(f[0]) > 0 :
            var = '%s' % f[0]
            selected_PDP_cplex[var]=(u"\u2588")
            print ' Facility %s is selected' % f[0]
    val_dispersion_cplex = solution.get_objective_value()
    VAL_DISPERSION_cplex.append(round(val_dispersion_cplex, 3))
    solve_time_cplex.append(t2CPLEX)
    
    print '**********************************************************************'
    print 'Largest Value in dij (M)     = ', M_cplex
    print 'Objective Value (D)          = ', solution.get_objective_value()
    print 'Candidate Facilities         = ', p_facilities
    print 'Matrix Dimensions            = ', dij.shape
    print 'Real Time to Solve (minutes) = ', t2CPLEX
    print 'Solution status              = ', solution.get_status(), ':', \
                                              solution.status[solution.get_status()]
    print '**********************************************************************'
    print '    -- The p-Dispersion Problem CPLEX -- '
    print '    -- James Gaboardi, 2016 -- '
    print ' [n] = ', str(n), '\n\n'
    
###############################################################################    
    
    #  Gurobi
    
    t1Gurobi = time.time()
    
    facility_range = range(total_facilities)     # range of total facilities
    M_gurobi = np.amax(dij)                             # Max Value in dij
    
    mPDP = gbp.Model(" -- p-Dispersion -- ")    # Instantiate a model
    gbp.setParam('MIPFocus', 2)                 # Set MIP Focus to 2 for optimality
    
    # Add Decision Variables
    facility_variable_GUROBI = []
    for destination in facility_range:
        facility_variable_GUROBI.append(mPDP.addVar(vtype=gbp.GRB.BINARY,
                                                lb=0,
                                                ub=1,
                                              name='y'+str(destination+1)))
    # Add Maximized Minimum Variable
    D_gurobi = mPDP.addVar(vtype=gbp.GRB.CONTINUOUS,
                       lb=0,
                       ub=gbp.GRB.INFINITY,
                     name='D')
    # Update Model Variables
    mPDP.update()       
    
    #  Set Objective Function
    mPDP.setObjective(D_gurobi, gbp.GRB.MAXIMIZE)
    
    # Add Facility Constraint
    mPDP.addConstr(gbp.quicksum(facility_variable_GUROBI[destination]                  \
                                                    for destination in facility_range) \
                                                    == p_facilities)                        
    
    # Add Inter-Facility Distance Constraints ==> n(n-1)/2
    for origin in facility_range:
        for destination in facility_range:
            if destination > origin:
                mPDP.addConstr(
                                dij[origin][destination]                             \
                                + M_gurobi * 2                                       \
                                - M_gurobi * facility_variable_GUROBI[origin]        \
                                - M_gurobi * facility_variable_GUROBI[destination]   \
                                >= D_gurobi)
            else:
                pass
    
    #  Optimize and Print Results
    mPDP.optimize()
    mPDP.write('path.lp')
    t2Gurobi = time.time()-t1Gurobi
    solve_time_gurobi.append(t2Gurobi)
    
    print '\n**********************************************************************'
    selected_facilities_gurobi = OrderedDict()
    for v in mPDP.getVars():
        if 'x' in v.VarName or 'D' in v.VarName:
            pass
        elif v.x > 0:
            var = '%s' % v.VarName
            selected_facilities_gurobi[var]=(u"\u2588")
            print 'Facility %s is selected' % v.x
    print '    | Selected Facility Locations -------------  ^^^^ '
    print '    | Candidate Facilities [p] ---------------- ', len(selected_facilities_gurobi)
    print '    | Largest Value in dij (M) ---------------- ', M_gurobi
    print '    | Objective Value (D) --------------------- ', mPDP.objVal
    print '    | Matrix Dimensions ----------------------- ', dij.shape
    print '    | Real Time to Solve (minutes)------------- ', t2Gurobi
    print '**********************************************************************'
    print '    -- The p-Dispersion Problem Gurobi -- '
    print '\n    -- James Gaboardi, 2016 -- '
    print ' [n] = ', str(n), '\n\n'
    
############################################################################################################  
    
# Data can be read-in or simulated

for n in range(5, N+5): #--------- n Matrix Dimensions

    #  Total Number of Facilities  
    Service = matrix_vector = n       # matrix_vector * matrix_vector for total facilities

    P = candidate_facilities = 2

    # Cost Matrix
    Cost_Matrix = np.random.randint(3, 
                                    50, 
                                    matrix_vector*matrix_vector)
    Cost_Matrix = Cost_Matrix.reshape(matrix_vector,matrix_vector)    


    

    Cplex_and_Gurobi_pDispersion(dij=Cost_Matrix, 
                                 p_facilities=P, 
                                 total_facilities=Service)

In [ ]:
print len(solve_time_cplex)
print len(solve_time_gurobi)

In [ ]:
solve_df.index.name = 'Matrix Size'
solve_df['CPLEX Solution Time (Real Time (sec.))'] = solve_time_cplex
solve_df['Gurobi Solution Time (Real Time (sec.))'] = solve_time_gurobi
qgrid.show_grid(solve_df)

In [ ]:
TOOLS = 'wheel_zoom, pan, reset, crosshair, save, hover'

source_cplex = ColumnDataSource(
        data=dict(
            x=range(1, N+5),
            y=solve_time_cplex,
            obj=solve_time_cplex,
            desc=n_list))

source_gurobi = ColumnDataSource(
        data=dict(
            x=range(1, N+1),
            y=solve_time_gurobi,
            obj=solve_time_gurobi,
            desc=n_list))


plot_solve_time = figure(title="Solution Time Comparison", 
                        plot_width=600, plot_height=400, 
                        tools=[TOOLS], y_range=(-1,2))

plot_solve_time.circle('x', 'y', size=5, color='red', source=source_cplex, legend='CPLEX')
plot_solve_time.line('x', 'y', 
                 color="#ff4d4d", alpha=0.2, line_width=2, source=source_cplex, legend='CPLEX')

plot_solve_time.circle('x', 'y', size=5, color='green', source=source_gurobi, legend='Gurobi')
plot_solve_time.line('x', 'y', 
                 color='#00b300', alpha=0.2, line_width=2, source=source_gurobi, legend='Gurobi')


plot_solve_time.xaxis.axis_label = '[p = n]'
plot_solve_time.yaxis.axis_label = 'Seconds'

one_quarter = BoxAnnotation(plot=plot_solve_time, top=.35, 
                            fill_alpha=0.1, fill_color='green')
half = BoxAnnotation(plot=plot_solve_time, bottom=.35, top=.7, 
                            fill_alpha=0.1, fill_color='blue')
three_quarter = BoxAnnotation(plot=plot_solve_time, bottom=.7, top=1.05,
                            fill_alpha=0.1, fill_color='gray')

plot_solve_time.renderers.extend([one_quarter, half, three_quarter])
show(plot_solve_time)